In [ ]:
#Takes DRS batch file (.csv) and returns absolute filepaths (.txt) for Handprint + OCR lookup table (.csv)
##Matt Cook - 2021
import pandas as pd
import os
import sys
from pathlib import Path

#inputs
tableIn = pd.read_csv(".../Nextcloud/Houghton/SAEF/saefdvtest/test_drs_batch.csv", usecols=['FILE-OSN']) ##batch info csv
target =".../Nextcloud/Houghton/SAEF/saefdvtest/test/data" #target image directory

#outputs
pathsOut = open(".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/HP_searchPaths.txt", "w") # for Handprint
tableOut = ".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/OCR_LookupTable.csv"# transcription batch csv

#dataframe
df = pd.DataFrame()

#cross-check that DRS FILE-OSN values exist in target directory and add matches to pathsOut + dataframe
for path in sorted(Path(target).rglob('*.jpg')):
    if path.stem in tableIn.values: #check if image exists in batch info
        absolute = (str(path.parent) + "/" + path.name) #absolute path for images
        pathsOut.write(str(absolute)) #write paths to pathsOut
        pathsOut.write("\n")
        df = df.append({'FILE-OSN':path.stem,'IMG-PATH':absolute}, ignore_index=True) #append data frame  
        print("image " + path.stem + " located " + "at " + absolute) #console out

#create new lookupt table from dataframe
with open(tableOut, mode = 'w') as f:
    df.to_csv(f,index=False) #append tableOut with FILE-OSN and IMG-PATH values

print("\n")
print("lookup table created for collection")
pathsOut.close()


In [ ]:
#Run Handprint using paths from local file
##Mike Hucka designed and implemented Handprint beginning in mid-2018.
##installation instructions at https://github.com/caltechlibrary/handprint

##generate Microsoft results
!handprint --service microsoft -@".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/OCR_log.txt" --from-file ".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/HP_searchPaths.txt" --no-grid --extended --output ".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/MSFT"

#generate Amazon results
#!handprint --service google -@"/Users/matthewcook/Dropbox/DS/Berry/Test 2/OCR_log.txt" --from-file "/Users/matthewcook/Dropbox/DS/Berry/Test 2/saefdvtest.txt" --no-grid --extended --output "/Users/matthewcook/Dropbox/DS/Berry/Test 2/HP Out"

#generate Google results
#!handprint --amazon-textract -@"/Users/matthewcook/Dropbox/DS/Berry/Test 2/OCR_log.txt" --from-file "/Users/matthewcook/Dropbox/DS/Berry/Test 2/saefdvtest.txt" --no-grid --extended --output "/Users/matthewcook/Dropbox/DS/Berry/Test 2/HP Out"


In [ ]:
#Takes preliminary lookup table (CSV) and outputs updated table with locations for Handprint outputs, by file type
##Matt Cook - 2021

import pandas as pd
import os
import sys
from pathlib import Path

#Declarations
table = ".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/OCR_LookupTable.csv" ##Lookup table
hpOut =".../Nextcloud/Houghton/SAEF/saefdvtest/OCR/MSFT"##HandPrint Outputs
types = ['MSFT-IMG','MSFT-JSON','MSFT-TXT']##additional lookup table columns
suffX = ['.png','.json','.txt']#OCR file types
num = 0 ##counter
df = [1, 2, 3] #empty dataframes

#match DRS "FILE-OSN" to OCR outputs and append new dataframes 
while num < len(df):  
    df[num] = pd.DataFrame()
    for path in sorted(Path(hpOut).rglob('*' + str(suffX[num]))): #Loop through file types
        #define location and match to DRS's "FILE-OSN" column value
        absolute = (str(path.parent) + "/" + path.name)
        match = path.stem.split('.')
        fileCore = pd.read_csv(table, usecols=['FILE-OSN'])
        #append dataframe with matching values
        for value in fileCore.values:
            if value == match[0]:
                df[num] = df[num].append({types[num]:absolute}, ignore_index=True)
                print("\n")
                print("dataframe " + str(num + 1) + " appended with " + suffX[num] + " located at " + absolute)
    num = num + 1   
    
#concatenate newly appended data frames with existing lookup table
conC = pd.concat(df, axis=1)
print("\n")
print("new dataframes concatenated")
out = pd.concat([pd.read_csv(table),conC], axis=1)

#update lookup table CSV
with open(table, mode = 'w') as f:
    out.to_csv(f,index=False, header=f.tell()==0) #append tableOut with FILE-OSN and IMG-PATH values
    print("\n")
    print("lookup table updated with HandPrint output locations for " + str(types) + "-type transcriptions")
    print("\n")
    print("have a nice day")
        